<a href="https://colab.research.google.com/github/duaneywadey/A-collection-of-Jupyter-notebooks-and-CSV-files/blob/collection-of-models/MovieRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [364]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [365]:
df = pd.read_csv('https://raw.githubusercontent.com/duaneywadey/A-collection-of-Jupyter-notebooks-and-CSV-files/main/CSV%20Files/movies.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [366]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [367]:
df.shape

(4803, 24)

In [368]:
# Selecting relevant features

selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [369]:
# Replacing null values

for feature in selected_features:
  df[feature] = df[feature].fillna('')

In [370]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [371]:
df.isnull().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

In [372]:
# CONCATENATING DF COLUMNS

dfList = []
combinedFeatures = ''
for s in selected_features:
  dfList.append(df[s])
  for item in dfList:
    combinedFeatures += item

print(combinedFeatures)

0       Action Adventure Fantasy Science FictionAction...
1       Adventure Fantasy ActionAdventure Fantasy Acti...
2       Action Adventure CrimeAction Adventure Crimesp...
3       Action Crime Drama ThrillerAction Crime Drama ...
4       Action Adventure Science FictionAction Adventu...
                              ...                        
4798    Action Crime ThrillerAction Crime Thrillerunit...
4799    Comedy RomanceComedy RomanceComedy RomanceA ne...
4800    Comedy Drama Romance TV MovieComedy Drama Roma...
4801    A New Yorker in ShanghaiA New Yorker in Shangh...
4802    DocumentaryDocumentaryobsession camcorder crus...
Name: genres, Length: 4803, dtype: object


In [373]:
# Converting data to feature vectors

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combinedFeatures)
feature_vectors

<4803x36490 sparse matrix of type '<class 'numpy.float64'>'
	with 126033 stored elements in Compressed Sparse Row format>

In [374]:
# Getting similarity scores

similarity = cosine_similarity(feature_vectors)
similarity

array([[1.        , 0.09504587, 0.02671215, ..., 0.        , 0.        ,
        0.        ],
       [0.09504587, 1.        , 0.04130295, ..., 0.02224845, 0.        ,
        0.        ],
       [0.02671215, 0.04130295, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.02224845, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [375]:
# Getting movie name from the user

movie_name = input('Enter your fav movie: ')


Enter your fav movie: zombie


In [376]:
# Creating a list of recommended movies

listOfTitles = df['title'].tolist()
listOfTitles

['Avatar',
 "Pirates of the Caribbean: At World's End",
 'Spectre',
 'The Dark Knight Rises',
 'John Carter',
 'Spider-Man 3',
 'Tangled',
 'Avengers: Age of Ultron',
 'Harry Potter and the Half-Blood Prince',
 'Batman v Superman: Dawn of Justice',
 'Superman Returns',
 'Quantum of Solace',
 "Pirates of the Caribbean: Dead Man's Chest",
 'The Lone Ranger',
 'Man of Steel',
 'The Chronicles of Narnia: Prince Caspian',
 'The Avengers',
 'Pirates of the Caribbean: On Stranger Tides',
 'Men in Black 3',
 'The Hobbit: The Battle of the Five Armies',
 'The Amazing Spider-Man',
 'Robin Hood',
 'The Hobbit: The Desolation of Smaug',
 'The Golden Compass',
 'King Kong',
 'Titanic',
 'Captain America: Civil War',
 'Battleship',
 'Jurassic World',
 'Skyfall',
 'Spider-Man 2',
 'Iron Man 3',
 'Alice in Wonderland',
 'X-Men: The Last Stand',
 'Monsters University',
 'Transformers: Revenge of the Fallen',
 'Transformers: Age of Extinction',
 'Oz: The Great and Powerful',
 'The Amazing Spider-Man 2',

In [377]:
# Find close match

findCloseMatch = difflib.get_close_matches(movie_name, listOfTitles)
findCloseMatch

['Zombieland', 'Womb', 'Home']

In [378]:
# Print a close match

closeMatch = findCloseMatch[0]
closeMatch

'Zombieland'

In [379]:
indexOfMovie = df[df.title == closeMatch]['index'].values[0]
indexOfMovie

1988

In [380]:
# Similarity scores

similarityScore = list(enumerate(similarity[indexOfMovie]))
similarityScore

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.013835809611491667),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.00971562386263821),
 (13, 0.0),
 (14, 0.0),
 (15, 0.15405321792588966),
 (16, 0.0),
 (17, 0.0),
 (18, 0.01430857661973631),
 (19, 0.0),
 (20, 0.024744725728055205),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.009214972853783207),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.039081534351731194),
 (33, 0.0),
 (34, 0.0),
 (35, 0.010396758970803876),
 (36, 0.009453660374027971),
 (37, 0.010786631668251894),
 (38, 0.026728474196197435),
 (39, 0.0),
 (40, 0.0026131457753964856),
 (41, 0.0),
 (42, 0.0),
 (43, 0.0),
 (44, 0.0),
 (45, 0.01753298050436357),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.008717411824364477),
 (50, 0.0),
 (51, 0.0),
 (52, 0.0),
 (53, 0.0),
 (54, 0.0),
 (55, 0.017071143299690512),
 (56, 0.0),
 (57, 0.0),
 (58, 0.019324201957397896),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0

In [381]:
# Sorting based on similarity score, lambda function takes second value

sortedSimilarityScore = sorted(similarityScore, key = lambda x:x[1], reverse=True)
sortedSimilarityScore

[(1988, 1.0),
 (3583, 0.29053068803756904),
 (3270, 0.29043835718405164),
 (2493, 0.23572392693555433),
 (1590, 0.2312877406430851),
 (1745, 0.22868841016406577),
 (3559, 0.22014243810851122),
 (3171, 0.2134423335686679),
 (2871, 0.2129282256646856),
 (1240, 0.21105034942409398),
 (1560, 0.20660121400597042),
 (4172, 0.2065546300442014),
 (3937, 0.2050259493305325),
 (4206, 0.20142443762424067),
 (1218, 0.19907579619349136),
 (2570, 0.19878942622940637),
 (4454, 0.19747467336412156),
 (3605, 0.19527181706486188),
 (1292, 0.19006773243636046),
 (2335, 0.18739805059753156),
 (817, 0.18573759779837892),
 (4254, 0.1845341702296029),
 (1322, 0.18171338332944909),
 (200, 0.177861768542395),
 (2004, 0.17744707497992754),
 (3178, 0.1731842031500856),
 (3370, 0.17300899242594742),
 (404, 0.17048149177131663),
 (4425, 0.16714374696297796),
 (2913, 0.15973141889441972),
 (2713, 0.15932892090881068),
 (3742, 0.15671901515578854),
 (2444, 0.15651601328342235),
 (1070, 0.15481455023887553),
 (15, 0.

In [387]:
# Titles of similar movies

print('MOVIES SUGGESTED: ')

i=1
for movie in sortedSimilarityScore:
  index = movie[0]
  indexTitle = df[df.index==index]['title'].values[0]
  if i<30:
    print(i,'.', indexTitle)
    i+=1
    

MOVIES SUGGESTED: 
1 . Zombieland
2 . Our Idiot Brother
3 . Howards End
4 . The Immigrant
5 . Atonement
6 . My Sister's Keeper
7 . Paranormal Activity 3
8 . Harold & Kumar Go to White Castle
9 . Running with Scissors
10 . The Other Boleyn Girl
11 . 27 Dresses
12 . A Streetcar Named Desire
13 . Happiness
14 . Caramel
15 . The Guilt Trip
16 . Ramona and Beezus
17 . Welcome to the Dollhouse
18 . Sunshine Cleaning
19 . Because I Said So
20 . Blue Jasmine
21 . American Wedding
22 . Mutant World
23 . City of Ember
24 . The Hunger Games: Mockingjay - Part 1
25 . Marvin's Room
26 . The Savages
27 . Rabbit-Proof Fence
28 . Memoirs of a Geisha
29 . Zombie Hunter


In [383]:
print(indexTitle)

My Date with Drew
